In [1]:
import pandas as pd
import os
import re
import tqdm
import matplotlib.pyplot as plt

class QF_NRA:
    count = 0
    def __init__(self, init=False, path='../../QF_NRA', data_path='result'):
        self.path = path
        self.data_path = data_path
        self.records = pd.read_excel(r"base.xlsx", header=[0], index_col = [0])
        if init:
            self.get_info(path)

    def get_info(self, path):
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                with open(new_path) as f:
                    vars = 0
                    assertions = 0
                    line = f.readline()
                    while line:
                        if line[:3] == '(de':
                            vars += 1
                        elif line[:3] == '(as':
                            assertions += 1
                        line = f.readline()
                    # print(count, "/", total,": ", filename, "+", vars, "+", assertions)
                    self.count += 1
                    new_path = new_path[6:]
                    category = new_path.split('/')[1]
                    self.records.loc[len(self.records.index)] = [new_path, category, vars, assertions, '0', '0']
                    # self.records = self.records.concat({'filename': new_path, 'category':category, 'vars':vars, 'assertions':assertions}, ignore_index=True, verify_integrity=True)

    def arrange(self):
        file_list = os.listdir(self.data_path)
        for filename in tqdm(file_list, desc = self.data_path):
            with open(os.path.join(self.data_path, filename)) as f:
                while f.readline():
                    result = f.readline()
                    if result[:3] == 'run':
                        result = 'dump'
                    info = f.readline()
                    name = info.split(' : ')[0]
                    timeout = info.split(' : ')[1][:-5]
                    self.records.loc[self.records['filename'] == name, ['result', 'timeout']] = [result, timeout]
        
    def save_to(self, filename):
        self.records.to_excel(filename)
    
    def show(self):
        folder_list = os.listdir(self.path)
        name = []
        solved = []
        count = []
        dump = []
        timeout = []
        for folder in folder_list:
            name.append(folder)
            count.append(len(self.records[self.records['category'] == folder]))
            solved.append(len(self.records[self.records['category'] == folder & self.records['result'].str.contains('sat')]))
            dump.append(len(self.records[self.records['category'] == folder & self.records['result'] == ('dump')]))
            timeout.append(len(self.records[self.records['category'] == folder & self.records['result'] == ('timeout')]))

        plt.bar(folder_list, name, label="name")
        plt.bar(folder_list, solved, label="solved")
        plt.bar(folder_list, count, label="count")
        plt.bar(folder_list, timeout, label="timeout")
        plt.show()

In [2]:
a = QF_NRA()

a.records

,filename,category,vars,assertions,result,timeout
0,QF_NRA/zankl/matrix-2-all-17.smt2,zankl,206,207,0,0
1,QF_NRA/zankl/matrix-4-all-15.smt2,zankl,215,216,0,0
2,QF_NRA/zankl/matrix-1-all-2.smt2,zankl,14,15,0,0
3,QF_NRA/zankl/matrix-5-all-27.smt2,zankl,137,138,0,0
4,QF_NRA/zankl/matrix-4-all-26.smt2,zankl,74,75,0,0
...,...,...,...,...,...,...
11547,QF_NRA/kissing/kissing_4_6.smt2,kissing,24,21,0,0
11548,QF_NRA/kissing/kissing_3_10.smt2,kissing,30,55,0,0
11549,QF_NRA/kissing/kissing_4_5.smt2,kissing,20,15,0,0
11550,QF_NRA/kissing/kissing_4_15.smt2,kissing,60,120,0,0
